In [ ]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import ee, geemap, json
# ee.Authenticate()
ee.Initialize()
from city_metrix.layers import Layer, EsaWorldCover, EsaWorldCoverClass

FILEPATH = 'C:/Users/theodore.wong/danida_africa'

In [ ]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

In [ ]:
def percent_builtup(geoms):
    built_up_land = EsaWorldCover(land_cover_class=EsaWorldCoverClass.BUILT_UP)
    built_land = built_up_land.groupby(geoms).count()
    all_land = EsaWorldCover().groupby(geoms).count()
    return built_land / all_land

In [ ]:
for fname in boundary_filenames:
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
            feature = geemap.geojson_to_ee(json_obj)
            geoms = geemap.ee_to_gdf(feature)['geometry']
            result = (percent_builtup(geoms) * 100).round(decimals=1)
            out_gdf = geemap.ee_to_gdf(feature)
            out_gdf['percent_builtup'] = result
            with open('{0}/percent_builtup-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
                ofile.write(out_gdf.to_json())
            print(cityname)